In [ ]:
  # importing the packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing

In [ ]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')# data visualisation and manipulationimport numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
import missingno as msno#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)#import the necessary modelling algos.
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

#model selection
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,precision_score
from sklearn.model_selection import GridSearchCV#preprocess.
from sklearn.preprocessing import MinMaxScaler,StandardScaler

In [ ]:
#importing the dataset
url="drive/MyDrive/NS/Lab/MachineLearningCSV/CSVs/dataset.csv"
df = pd.read_csv(url)

In [ ]:
print(df.shape)
df1=df.drop(['Unnamed: 0'], axis =1)
N=1000000
df2=df1.groupby('Label', group_keys=False).apply(lambda x: x.sample(int(np.rint(N*len(x)/len(df1))))).sample(frac=1).reset_index(drop=True)

(2830743, 80)


In [ ]:
df1["Label"].value_counts()

BENIGN                        2273097
DoS Hulk                       231073
PortScan                       158930
DDoS                           128027
DoS GoldenEye                   10293
FTP-Patator                      7938
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1966
Web Attack � Brute Force         1507
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Heartbleed                         11
Name: Label, dtype: int64

In [ ]:
df2

,Destination_Port,Flow_Duration,Total_Fwd_Packets,Total_Backward_Packets,Total_Length_of_Fwd_Packets,Total_Length_of_Bwd_Packets,Fwd_Packet_Length_Max,Fwd_Packet_Length_Min,Fwd_Packet_Length_Mean,Fwd_Packet_Length_Std,Bwd_Packet_Length_Max,Bwd_Packet_Length_Min,Bwd_Packet_Length_Mean,Bwd_Packet_Length_Std,Flow_Bytess,Flow_Packetss,Flow_IAT_Mean,Flow_IAT_Std,Flow_IAT_Max,Flow_IAT_Min,Fwd_IAT_Total,Fwd_IAT_Mean,Fwd_IAT_Std,Fwd_IAT_Max,Fwd_IAT_Min,Bwd_IAT_Total,Bwd_IAT_Mean,Bwd_IAT_Std,Bwd_IAT_Max,Bwd_IAT_Min,Fwd_PSH_Flags,Bwd_PSH_Flags,Fwd_URG_Flags,Bwd_URG_Flags,Fwd_Header_Length,Bwd_Header_Length,Fwd_Packetss,Bwd_Packetss,Min_Packet_Length,Max_Packet_Length,Packet_Length_Mean,Packet_Length_Std,Packet_Length_Variance,FIN_Flag_Count,SYN_Flag_Count,RST_Flag_Count,PSH_Flag_Count,ACK_Flag_Count,URG_Flag_Count,CWE_Flag_Count,ECE_Flag_Count,DownUp_Ratio,Average_Packet_Size,Avg_Fwd_Segment_Size,Avg_Bwd_Segment_Size,Fwd_Header_Length.1,Fwd_Avg_BytesBulk,Fwd_Avg_PacketsBulk,Fwd_Avg_Bulk_Rate,Bwd_Avg_BytesBulk,Bwd_Avg_PacketsBulk,Bwd_Avg_Bulk_Rate,Subflow_Fwd_Packets,Subflow_Fwd_Bytes,Subflow_Bwd_Packets,Subflow_Bwd_Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active_Mean,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Label
0,7921,48,1,1,2,6,2,2,2.000,0.000000,6,6,6.00,0.000000,1.666667e+05,41666.666670,4.800000e+01,0.000000e+00,48,48,0,0.000000e+00,0.000000e+00,0,0,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,24,20,20833.333330,20833.333330,2,6,3.333333,2.309401,5.333333e+00,0,0,0,1,0,0,0,0,1,5.000000,2.000,6.00,24,0,0,0,0,0,0,1,2,1,6,1024,0,0,24,0.0,0.0,0,0,0.0,0.0,0,0,PortScan
1,80,84733276,8,5,360,11595,354,0,45.000,124.872506,7240,0,2319.00,3331.638186,1.410898e+02,0.153423,7.061106e+06,2.440000e+07,84600000,1,84600000,1.210000e+07,3.200000e+07,84600000,1,154628,3.865700e+04,6.802637e+04,140227,190,0,0,0,0,252,168,0.094414,0.059009,0,7240,853.928571,2169.804900,4.708053e+06,0,0,0,0,1,0,0,0,0,919.615385,45.000,2319.00,252,0,0,0,0,0,0,8,360,5,11595,274,235,2,20,1963.0,0.0,1963,1963,84600000.0,0.0,84600000,84600000,DoS Hulk
2,53,605070,1,1,55,55,55,55,55.000,0.000000,55,55,55.00,0.000000,1.817971e+02,3.305403,6.050700e+05,0.000000e+00,605070,605070,0,0.000000e+00,0.000000e+00,0,0,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,20,20,1.652701,1.652701,55,55,55.000000,0.000000,0.000000e+00,0,0,0,0,0,0,0,0,1,82.500000,55.000,55.00,20,0,0,0,0,0,0,1,55,1,55,-1,-1,0,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,443,10137210,8,10,1441,5888,1098,0,180.125,377.834596,1689,2,588.80,719.382729,7.229800e+02,1.775636,5.963065e+05,2.423190e+06,9999540,0,137668,1.966686e+04,2.050020e+04,52477,3,10104932,1.122770e+06,3.328869e+06,9999632,2,0,0,0,0,172,204,0.789172,0.986465,0,1689,385.736842,603.520491,3.642370e+05,0,0,0,1,0,0,0,0,1,407.166667,180.125,588.80,172,0,0,0,0,0,0,8,1441,10,5888,8192,31842,7,20,137668.0,0.0,137668,137668,9999540.0,0.0,9999540,9999540,BENIGN
4,443,38797,1,1,6,6,6,6,6.000,0.000000,6,6,6.00,0.000000,3.093023e+02,51.550378,3.879700e+04,0.000000e+00,38797,38797,0,0.000000e+00,0.000000e+00,0,0,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,20,20,25.775189,25.775189,6,6,6.000000,0.000000,0.000000e+00,0,0,0,0,1,1,0,0,1,9.000000,6.000,6.00,20,0,0,0,0,0,0,1,6,1,6,320,5120,0,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,1862,105,2,2,4,12,2,2,2.000,0.000000,6,6,6.00,0.000000,1.523810e+05,38095.238100,3.500000e+01,2.253886e+01,49,9,49,4.900000e+01,0.000000e+00,49,49,47,4.700000e+01,0.000000e+00,47,47,0,0,0,0,48,40,19047.619050,19047.619050,2,6,3.600000,2.190890,4.800000e+00,0,0,0,1,0,0,0,0,1,4.500000,2.000,6.00,48,0,0,0,0,0,0,2,4,2,12,1024,0,1,24,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
999996,80,7097

In [ ]:
df2.shape

(1000000, 79)

In [ ]:
df2.groupby(['Label']).transform('count').nunique()

Destination_Port               15
Flow_Duration                  15
Total_Fwd_Packets              15
Total_Backward_Packets         15
Total_Length_of_Fwd_Packets    15
                               ..
Active_Min                     15
Idle_Mean                      15
Idle_Std                       15
Idle_Max                       15
Idle_Min                       15
Length: 78, dtype: int64

In [ ]:
# Replacing infinite with nan
df2.replace([np.inf, -np.inf], np.nan, inplace=True)

# Dropping all the rows with nan values
df2.dropna(inplace=True)

# Printing df
df2

,Destination_Port,Flow_Duration,Total_Fwd_Packets,Total_Backward_Packets,Total_Length_of_Fwd_Packets,Total_Length_of_Bwd_Packets,Fwd_Packet_Length_Max,Fwd_Packet_Length_Min,Fwd_Packet_Length_Mean,Fwd_Packet_Length_Std,Bwd_Packet_Length_Max,Bwd_Packet_Length_Min,Bwd_Packet_Length_Mean,Bwd_Packet_Length_Std,Flow_Bytess,Flow_Packetss,Flow_IAT_Mean,Flow_IAT_Std,Flow_IAT_Max,Flow_IAT_Min,Fwd_IAT_Total,Fwd_IAT_Mean,Fwd_IAT_Std,Fwd_IAT_Max,Fwd_IAT_Min,Bwd_IAT_Total,Bwd_IAT_Mean,Bwd_IAT_Std,Bwd_IAT_Max,Bwd_IAT_Min,Fwd_PSH_Flags,Bwd_PSH_Flags,Fwd_URG_Flags,Bwd_URG_Flags,Fwd_Header_Length,Bwd_Header_Length,Fwd_Packetss,Bwd_Packetss,Min_Packet_Length,Max_Packet_Length,Packet_Length_Mean,Packet_Length_Std,Packet_Length_Variance,FIN_Flag_Count,SYN_Flag_Count,RST_Flag_Count,PSH_Flag_Count,ACK_Flag_Count,URG_Flag_Count,CWE_Flag_Count,ECE_Flag_Count,DownUp_Ratio,Average_Packet_Size,Avg_Fwd_Segment_Size,Avg_Bwd_Segment_Size,Fwd_Header_Length.1,Fwd_Avg_BytesBulk,Fwd_Avg_PacketsBulk,Fwd_Avg_Bulk_Rate,Bwd_Avg_BytesBulk,Bwd_Avg_PacketsBulk,Bwd_Avg_Bulk_Rate,Subflow_Fwd_Packets,Subflow_Fwd_Bytes,Subflow_Bwd_Packets,Subflow_Bwd_Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active_Mean,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Label
0,80,109670936,4,3,360,0,351,0,90.000000,174.051716,0,0,0.00,0.00000,3.282547e+00,0.063827,1.830000e+07,4.480000e+07,110000000,68,110000000,36600000.0,6.330000e+07,110000000,628,110000000,5.480000e+07,7.750000e+07,110000000,5230,0,0,0,0,136,104,0.036473,0.027355,0,351,45.000000,123.682774,1.529743e+04,0,0,0,1,0,0,0,0,0,51.428571,90.000000,0.00,136,0,0,0,0,0,0,4,360,3,0,29200,235,2,32,5334.0,0.0,5334,5334,110000000.0,0.0,110000000,110000000,DoS Slowhttptest
1,53,297,2,2,88,344,44,44,44.000000,0.000000,172,172,172.00,0.00000,1.454545e+06,13468.013468,9.900000e+01,1.283394e+02,245,4,4,4.0,0.000000e+00,4,4,48,4.800000e+01,0.000000e+00,48,48,0,0,0,0,40,40,6734.006734,6734.006734,44,172,95.200000,70.108487,4.915200e+03,0,0,0,0,0,0,0,0,1,119.000000,44.000000,172.00,40,0,0,0,0,0,0,2,88,2,344,-1,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,53,244,2,2,78,340,39,39,39.000000,0.000000,170,170,170.00,0.00000,1.713115e+06,16393.442620,8.133333e+01,1.391414e+02,242,1,1,1.0,0.000000e+00,1,1,1,1.000000e+00,0.000000e+00,1,1,0,0,0,0,64,64,8196.721311,8196.721311,39,170,91.400000,71.751655,5.148300e+03,0,0,0,0,0,0,0,0,1,114.250000,39.000000,170.00,64,0,0,0,0,0,0,2,78,2,340,-1,-1,1,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,443,94900,1,1,0,0,0,0,0.000000,0.000000,0,0,0.00,0.00000,0.000000e+00,21.074816,9.490000e+04,0.000000e+00,94900,94900,0,0.0,0.000000e+00,0,0,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,32,32,10.537408,10.537408,0,0,0.000000,0.000000,0.000000e+00,0,0,0,0,1,1,0,0,1,0.000000,0.000000,0.00,32,0,0,0,0,0,0,1,0,1,0,304,254,0,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,80,73239802,6,4,44,11607,20,0,7.333333,6.653320,11595,0,2901.75,5795.50069,1.590802e+02,0.136538,8.137756e+06,2.140000e+07,64900000,6,71400000,14300000.0,2.850000e+07,64900000,63,1799483,5.998277e+05,1.038096e+06,1798518,186,0,0,0,0,132,92,0.081923,0.054615,0,11595,1059.727273,3494.158499,1.220000e+07,0,0,0,0,1,0,0,0,0,1165.700000,7.333333,2901.75,132,0,0,0,0,0,0,6,44,4,11607,256,229,4,20,781.0,0.0,781,781,35700000.0,41300000.0,64900000,6502424,DDoS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,3312,28,3,1,43,6,31,6,14.333333,14.433757,6,6,6.00,0.00000,1.750000e+06,142857.142900,9.333333e+00,1.443376e+01,26,1,2,1.0,0.000000e+00,1,1,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,60,20,107142.857100,35714.285710,6,31,11.000000,11.180340,1.250000e+02,0,0,0,0,1,0,0,0,0,13.750000,14.333333,6.00,60,0,0,0,0,0,0,3,43,1,6,1013,0,2,20,0.0,0.0,0,0,0.0,0.0,0,0

In [ ]:
df2=df2.drop(['Destination_Port'],axis=1)

In [ ]:
df2.shape

(998993, 78)

In [ ]:
df2["Label"].value_counts()

BENIGN                        802346
DoS Hulk                       81296
PortScan                       56093
DDoS                           45226
DoS GoldenEye                   3636
FTP-Patator                     2803
SSH-Patator                     2083
DoS slowloris                   2048
DoS Slowhttptest                1943
Bot                              693
Web Attack � Brute Force         532
Web Attack � XSS                 230
Infiltration                      13
Web Attack � Sql Injection         7
Heartbleed                         4
Name: Label, dtype: int64

In [ ]:
df2.isnull().values.all()

False

In [ ]:
df2

,Flow_Duration,Total_Fwd_Packets,Total_Backward_Packets,Total_Length_of_Fwd_Packets,Total_Length_of_Bwd_Packets,Fwd_Packet_Length_Max,Fwd_Packet_Length_Min,Fwd_Packet_Length_Mean,Fwd_Packet_Length_Std,Bwd_Packet_Length_Max,Bwd_Packet_Length_Min,Bwd_Packet_Length_Mean,Bwd_Packet_Length_Std,Flow_Bytess,Flow_Packetss,Flow_IAT_Mean,Flow_IAT_Std,Flow_IAT_Max,Flow_IAT_Min,Fwd_IAT_Total,Fwd_IAT_Mean,Fwd_IAT_Std,Fwd_IAT_Max,Fwd_IAT_Min,Bwd_IAT_Total,Bwd_IAT_Mean,Bwd_IAT_Std,Bwd_IAT_Max,Bwd_IAT_Min,Fwd_PSH_Flags,Bwd_PSH_Flags,Fwd_URG_Flags,Bwd_URG_Flags,Fwd_Header_Length,Bwd_Header_Length,Fwd_Packetss,Bwd_Packetss,Min_Packet_Length,Max_Packet_Length,Packet_Length_Mean,Packet_Length_Std,Packet_Length_Variance,FIN_Flag_Count,SYN_Flag_Count,RST_Flag_Count,PSH_Flag_Count,ACK_Flag_Count,URG_Flag_Count,CWE_Flag_Count,ECE_Flag_Count,DownUp_Ratio,Average_Packet_Size,Avg_Fwd_Segment_Size,Avg_Bwd_Segment_Size,Fwd_Header_Length.1,Fwd_Avg_BytesBulk,Fwd_Avg_PacketsBulk,Fwd_Avg_Bulk_Rate,Bwd_Avg_BytesBulk,Bwd_Avg_PacketsBulk,Bwd_Avg_Bulk_Rate,Subflow_Fwd_Packets,Subflow_Fwd_Bytes,Subflow_Bwd_Packets,Subflow_Bwd_Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active_Mean,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Label
0,156,2,0,12,0,6,6,6.000000,0.000000,0,0,0.000000,0.000000,7.692308e+04,12820.512820,1.560000e+02,0.000000e+00,156,156,156,156.000,0.000000e+00,156,156,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,40,0,12820.512820,0.000000,6,6,6.000000,0.000000,0.000000,0,0,0,0,1,0,0,0,0,9.000000,6.000000,0.000000,40,0,0,0,0,0,0,2,12,0,0,256,-1,1,20,0.0,0.000,0,0,0.0,0.0,0,0,BENIGN
1,85365947,20,20,1618,6190,427,0,80.900000,117.724880,1410,0,309.500000,464.204639,9.146504e+01,0.468571,2.188870e+06,1.010000e+07,58500000,1,85400000,4492944.579,1.430000e+07,58500000,26,85300000,4.488439e+06,1.430000e+07,58600000,1,0,0,0,0,424,412,0.234285,0.234285,0,1410,190.439024,350.617673,122932.752400,0,0,0,1,0,0,0,0,1,195.200000,80.900000,309.500000,424,0,0,0,0,0,0,20,1618,20,6190,8192,132,18,20,620266.5,756234.439,1155005,85528,42000000.0,23400000.0,58500000,25500000,BENIGN
2,150,2,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000e+00,13333.333330,1.500000e+02,0.000000e+00,150,150,150,150.000,0.000000e+00,150,150,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,40,0,13333.333330,0.000000,0,0,0.000000,0.000000,0.000000,0,0,0,0,1,0,0,0,0,0.000000,0.000000,0.000000,40,0,0,0,0,0,0,2,0,0,0,255,-1,0,20,0.0,0.000,0,0,0.0,0.0,0,0,BENIGN
3,3,2,0,4,0,2,2,2.000000,0.000000,0,0,0.000000,0.000000,1.333333e+06,666666.666700,3.000000e+00,0.000000e+00,3,3,3,3.000,0.000000e+00,3,3,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,48,0,666666.666700,0.000000,2,2,2.000000,0.000000,0.000000,0,0,0,1,0,0,0,0,0,3.000000,2.000000,0.000000,48,0,0,0,0,0,0,2,4,0,0,1024,-1,1,24,0.0,0.000,0,0,0.0,0.0,0,0,BENIGN
4,8584262,9,15,105,188,23,0,11.666667,9.354143,34,0,12.533333,14.549750,3.413223e+01,2.795814,3.732288e+05,9.772724e+05,3143363,3,6319547,789943.375,1.456589e+06,3189621,4,8584140,6.131529e+05,1.207415e+06,3143363,3,0,0,0,0,296,488,1.048430,1.747384,0,34,11.720000,12.601323,158.793333,0,0,0,1,0,0,0,0,1,12.208333,11.666667,12.533333,296,0,0,0,0,0,0,9,105,15,188,29200,227,6,32,0.0,0.000,0,0,0.0,0.0,0,0,FTP-Patator
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,5566006,3,1,12,0,6,0,4.000000,3.464102,0,0,0.000000,0.000000,2.155944e+00,0.718648,1.855335e+06,3.169759e+06,5515342,71,5566006,2783003.000,3.864111e+06,5515342,50664,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,72,32,0.538986,0.179662,0,6,2.400000,3.286335,10.800000,0,0,0,1,0,0,0,0,0,3.000000,4.000000,0.000000,72,0,0,0,0,0,0,3,12,1,0,8192,26883,2,20,0.0,0.000,0,0,0.0,0.0,0,0,BENIGN
999996,464,1,1,48,108,48,48,4

In [ ]:
X = df2.drop(['Label'], axis=1)
X.head()

,Flow_Duration,Total_Fwd_Packets,Total_Backward_Packets,Total_Length_of_Fwd_Packets,Total_Length_of_Bwd_Packets,Fwd_Packet_Length_Max,Fwd_Packet_Length_Min,Fwd_Packet_Length_Mean,Fwd_Packet_Length_Std,Bwd_Packet_Length_Max,Bwd_Packet_Length_Min,Bwd_Packet_Length_Mean,Bwd_Packet_Length_Std,Flow_Bytess,Flow_Packetss,Flow_IAT_Mean,Flow_IAT_Std,Flow_IAT_Max,Flow_IAT_Min,Fwd_IAT_Total,Fwd_IAT_Mean,Fwd_IAT_Std,Fwd_IAT_Max,Fwd_IAT_Min,Bwd_IAT_Total,Bwd_IAT_Mean,Bwd_IAT_Std,Bwd_IAT_Max,Bwd_IAT_Min,Fwd_PSH_Flags,Bwd_PSH_Flags,Fwd_URG_Flags,Bwd_URG_Flags,Fwd_Header_Length,Bwd_Header_Length,Fwd_Packetss,Bwd_Packetss,Min_Packet_Length,Max_Packet_Length,Packet_Length_Mean,Packet_Length_Std,Packet_Length_Variance,FIN_Flag_Count,SYN_Flag_Count,RST_Flag_Count,PSH_Flag_Count,ACK_Flag_Count,URG_Flag_Count,CWE_Flag_Count,ECE_Flag_Count,DownUp_Ratio,Average_Packet_Size,Avg_Fwd_Segment_Size,Avg_Bwd_Segment_Size,Fwd_Header_Length.1,Fwd_Avg_BytesBulk,Fwd_Avg_PacketsBulk,Fwd_Avg_Bulk_Rate,Bwd_Avg_BytesBulk,Bwd_Avg_PacketsBulk,Bwd_Avg_Bulk_Rate,Subflow_Fwd_Packets,Subflow_Fwd_Bytes,Subflow_Bwd_Packets,Subflow_Bwd_Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active_Mean,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min
0,109670936,4,3,360,0,351,0,90.000000,174.051716,0,0,0.00,0.00000,3.282547e+00,0.063827,1.830000e+07,4.480000e+07,110000000,68,110000000,36600000.0,63300000.0,110000000,628,110000000,5.480000e+07,7.750000e+07,110000000,5230,0,0,0,0,136,104,0.036473,0.027355,0,351,45.000000,123.682774,1.529743e+04,0,0,0,1,0,0,0,0,0,51.428571,90.000000,0.00,136,0,0,0,0,0,0,4,360,3,0,29200,235,2,32,5334.0,0.0,5334,5334,110000000.0,0.0,110000000,110000000
1,297,2,2,88,344,44,44,44.000000,0.000000,172,172,172.00,0.00000,1.454545e+06,13468.013468,9.900000e+01,1.283394e+02,245,4,4,4.0,0.0,4,4,48,4.800000e+01,0.000000e+00,48,48,0,0,0,0,40,40,6734.006734,6734.006734,44,172,95.200000,70.108487,4.915200e+03,0,0,0,0,0,0,0,0,1,119.000000,44.000000,172.00,40,0,0,0,0,0,0,2,88,2,344,-1,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0
2,244,2,2,78,340,39,39,39.000000,0.000000,170,170,170.00,0.00000,1.713115e+06,16393.442620,8.133333e+01,1.391414e+02,242,1,1,1.0,0.0,1,1,1,1.000000e+00,0.000000e+00,1,1,0,0,0,0,64,64,8196.721311,8196.721311,39,170,91.400000,71.751655,5.148300e+03,0,0,0,0,0,0,0,0,1,114.250000,39.000000,170.00,64,0,0,0,0,0,0,2,78,2,340,-1,-1,1,32,0.0,0.0,0,0,0.0,0.0,0,0
3,94900,1,1,0,0,0,0,0.000000,0.000000,0,0,0.00,0.00000,0.000000e+00,21.074816,9.490000e+04,0.000000e+00,94900,94900,0,0.0,0.0,0,0,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,32,32,10.537408,10.537408,0,0,0.000000,0.000000,0.000000e+00,0,0,0,0,1,1,0,0,1,0.000000,0.000000,0.00,32,0,0,0,0,0,0,1,0,1,0,304,254,0,32,0.0,0.0,0,0,0.0,0.0,0,0
4,73239802,6,4,44,11607,20,0,7.333333,6.653320,11595,0,2901.75,5795.50069,1.590802e+02,0.136538,8.137756e+06,2.140000e+07,64900000,6,71400000,14300000.0,28500000.0,64900000,63,1799483,5.998277e+05,1.038096e+06,1798518,186,0,0,0,0,132,92,0.081923,0.054615,0,11595,1059.727273,3494.158499,1.220000e+07,0,0,0,0,1,0,0,0,0,1165.700000,7.333333,2901.75,132,0,0,0,0,0,0,6,44,4,11607,256,229,4,20,781.0,0.0,781,781,35700000.0,41300000.0,64900000,6502424


In [ ]:
for col in X:
  print("col name: ",col," no of unique classes: ",X[col].nunique())

col name:  Flow_Duration  no of unique classes:  425050
col name:  Total_Fwd_Packets  no of unique classes:  913
col name:  Total_Backward_Packets  no of unique classes:  1127
col name:  Total_Length_of_Fwd_Packets  no of unique classes:  11614
col name:  Total_Length_of_Bwd_Packets  no of unique classes:  34178
col name:  Fwd_Packet_Length_Max  no of unique classes:  4242
col name:  Fwd_Packet_Length_Min  no of unique classes:  288
col name:  Fwd_Packet_Length_Mean  no of unique classes:  51309
col name:  Fwd_Packet_Length_Std  no of unique classes:  106121
col name:  Bwd_Packet_Length_Max  no of unique classes:  4304
col name:  Bwd_Packet_Length_Min  no of unique classes:  485
col name:  Bwd_Packet_Length_Mean  no of unique classes:  69919
col name:  Bwd_Packet_Length_Std  no of unique classes:  102279
col name:  Flow_Bytess  no of unique classes:  597343
col name:  Flow_Packetss  no of unique classes:  484146
col name:  Flow_IAT_Mean  no of unique classes:  458082
col name:  Flow_IA

In [ ]:
X.shape

(998993, 77)

In [ ]:
y = df2['Label']
y.head()
print("label no of unique classes: ", y.nunique())

label no of unique classes:  15


In [ ]:
y.shape

(998993,)

In [ ]:
y

0           BENIGN
1           BENIGN
2           BENIGN
3           BENIGN
4           BENIGN
            ...   
999995      BENIGN
999996    PortScan
999997      BENIGN
999998      BENIGN
999999    DoS Hulk
Name: Label, Length: 998943, dtype: object

In [ ]:
# Integer Encoding
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
print(y)

[ 5  0  0 ... 10  0  0]


In [ ]:
y1= pd.DataFrame(y)

In [ ]:
y1.shape

(999028, 1)

In [ ]:
y1

,0
0,10
1,4
2,0
3,0
4,0
...,...
998948,0
998949,2
998950,0
998951,0


In [ ]:
# sns.countplot(data=y1)
# y1.value_counts()

In [ ]:
# one hot encoding for y
# Integer Encoding
label_encoder = LabelEncoder()
y_label = label_encoder.fit_transform(y)
print(y_label)


#reshaping for OneHotEncoder
y_label_reshape = y_label.reshape(len(y_label), 1)

# One-Hot Encoding
onehot_encoder = OneHotEncoder(sparse=False)
y_one = onehot_encoder.fit_transform(y_label_reshape)
print(y_one)

[0 0 0 ... 0 0 0]
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [ ]:
# Normalizing numerical columns
# define standard scaler
scaler = preprocessing.MinMaxScaler()
# transform data
for col in X:
  print("normalized feature using Minmax scaler: ",col)
  X[col]=scaler.fit_transform(pd.to_numeric(X[col],errors='coerce').values.reshape(-1,1))

normalized feature using Minmax scaler:  Flow_Duration
normalized feature using Minmax scaler:  Total_Fwd_Packets
normalized feature using Minmax scaler:  Total_Backward_Packets
normalized feature using Minmax scaler:  Total_Length_of_Fwd_Packets
normalized feature using Minmax scaler:  Total_Length_of_Bwd_Packets
normalized feature using Minmax scaler:  Fwd_Packet_Length_Max
normalized feature using Minmax scaler:  Fwd_Packet_Length_Min
normalized feature using Minmax scaler:  Fwd_Packet_Length_Mean
normalized feature using Minmax scaler:  Fwd_Packet_Length_Std
normalized feature using Minmax scaler:  Bwd_Packet_Length_Max
normalized feature using Minmax scaler:  Bwd_Packet_Length_Min
normalized feature using Minmax scaler:  Bwd_Packet_Length_Mean
normalized feature using Minmax scaler:  Bwd_Packet_Length_Std
normalized feature using Minmax scaler:  Flow_Bytess
normalized feature using Minmax scaler:  Flow_Packetss
normalized feature using Minmax scaler:  Flow_IAT_Mean
normalized feat

In [ ]:
#pd.DataFrame(y_one, columns=['predictions']).to_csv('y_one.csv')

In [ ]:
X.shape

(999028, 77)

In [ ]:
#saving dataframe to csv in the drive
X.to_csv(r'drive/MyDrive/NS/Lab/MachineLearningCSV/CSVs/X_prepro.csv')

In [ ]:
X50 = X[['Average_Packet_Size','Packet_Length_Std', 'Packet_Length_Mean', 'Packet_Length_Variance', 'Subflow_Bwd_Bytes','Total_Length_of_Bwd_Packets','Avg_Bwd_Segment_Size','Bwd_Packet_Length_Mean','Init_Win_bytes_forward','Subflow_Fwd_Bytes','Total_Length_of_Fwd_Packets','Bwd_Packet_Length_Max','Max_Packet_Length','Init_Win_bytes_backward','Fwd_Packet_Length_Max','Flow_IAT_Max','Flow_Bytess','Flow_Duration','Fwd_IAT_Max','Fwd_Header_Length','Fwd_Header_Length.1','Avg_Fwd_Segment_Size','Fwd_Packet_Length_Mean','Fwd_Packetss','Bwd_Packetss','Fwd_IAT_Total','Flow_Packetss','Bwd_Header_Length','Flow_IAT_Mean','Fwd_IAT_Mean','Bwd_Packet_Length_Std','Flow_IAT_Std','Fwd_Packet_Length_Std','Fwd_IAT_Std','Bwd_IAT_Max','Subflow_Bwd_Packets','Total_Backward_Packets','Bwd_IAT_Total','Subflow_Fwd_Packets','Total_Fwd_Packets','Bwd_IAT_Mean','Bwd_Packet_Length_Min','Idle_Max','min_seg_size_forward','Idle_Mean','Idle_Min','Bwd_IAT_Std','Fwd_IAT_Min','Active_Mean']]


In [ ]:
X40 = X[['Average_Packet_Size','Packet_Length_Std', 'Packet_Length_Mean', 'Packet_Length_Variance', 'Subflow_Bwd_Bytes','Total_Length_of_Bwd_Packets','Avg_Bwd_Segment_Size','Bwd_Packet_Length_Mean','Init_Win_bytes_forward','Subflow_Fwd_Bytes','Total_Length_of_Fwd_Packets','Bwd_Packet_Length_Max','Max_Packet_Length','Init_Win_bytes_backward','Fwd_Packet_Length_Max','Flow_IAT_Max','Flow_Bytess','Flow_Duration','Fwd_IAT_Max','Fwd_Header_Length','Fwd_Header_Length.1','Avg_Fwd_Segment_Size','Fwd_Packet_Length_Mean','Fwd_Packetss','Bwd_Packetss','Fwd_IAT_Total','Flow_Packetss','Bwd_Header_Length','Flow_IAT_Mean','Fwd_IAT_Mean','Bwd_Packet_Length_Std','Flow_IAT_Std','Fwd_Packet_Length_Std','Fwd_IAT_Std','Bwd_IAT_Max','Subflow_Bwd_Packets','Total_Backward_Packets','Bwd_IAT_Total','Subflow_Fwd_Packets']]

In [ ]:
X50

,Average_Packet_Size,Packet_Length_Std,Packet_Length_Mean,Packet_Length_Variance,Subflow_Bwd_Bytes,Total_Length_of_Bwd_Packets,Avg_Bwd_Segment_Size,Bwd_Packet_Length_Mean,Init_Win_bytes_forward,Subflow_Fwd_Bytes,Total_Length_of_Fwd_Packets,Bwd_Packet_Length_Max,Max_Packet_Length,Init_Win_bytes_backward,Fwd_Packet_Length_Max,Flow_IAT_Max,Flow_Bytess,Flow_Duration,Fwd_IAT_Max,Fwd_Header_Length,Fwd_Header_Length.1,Avg_Fwd_Segment_Size,Fwd_Packet_Length_Mean,Fwd_Packetss,Bwd_Packetss,Fwd_IAT_Total,Flow_Packetss,Bwd_Header_Length,Flow_IAT_Mean,Fwd_IAT_Mean,Bwd_Packet_Length_Std,Flow_IAT_Std,Fwd_Packet_Length_Std,Fwd_IAT_Std,Bwd_IAT_Max,Subflow_Bwd_Packets,Total_Backward_Packets,Bwd_IAT_Total,Subflow_Fwd_Packets,Total_Fwd_Packets,Bwd_IAT_Mean,Bwd_Packet_Length_Min,Idle_Max,min_seg_size_forward,Idle_Mean,Idle_Min,Bwd_IAT_Std,Fwd_IAT_Min,Active_Mean
0,0.013960,0.026140,0.016596,6.829209e-04,0.000000e+00,0.000000e+00,0.000000,0.000000,0.445572,1.256058e-04,1.256058e-04,0.000000,0.014142,0.003601,0.014142,9.166667e-01,0.111921,9.139245e-01,9.166667e-01,0.99952,0.99952,0.016186,0.016186,1.215758e-08,1.367728e-08,9.166667e-01,0.400000,0.994604,1.525001e-01,3.050000e-01,0.000000,5.283019e-01,0.024426,7.482010e-01,9.166667e-01,0.000010,0.000010,9.166667e-01,0.000014,0.000014,4.566667e-01,0.000000,0.916667,1.0,0.916667,0.916667,0.929257,5.333333e-06,0.000052
1,0.032302,0.014817,0.035111,2.194286e-04,5.366603e-07,5.366603e-07,0.039353,0.039353,0.000000,3.070364e-05,3.070364e-05,0.009899,0.006930,0.000000,0.001773,2.150000e-06,0.112545,2.583333e-06,3.333333e-08,0.99952,0.99952,0.007913,0.007913,2.244669e-03,3.367003e-03,3.333333e-08,0.402694,0.994604,9.333332e-07,3.333333e-08,0.000000,1.513436e-06,0.000000,0.000000e+00,4.000000e-07,0.000007,0.000007,4.000000e-07,0.000005,0.000005,4.000000e-07,0.093074,0.000000,1.0,0.000000,0.000000,0.000000,1.333333e-07,0.000000
2,0.031012,0.015165,0.033709,2.298348e-04,5.304200e-07,5.304200e-07,0.038895,0.038895,0.000000,2.721459e-05,2.721459e-05,0.009784,0.006849,0.000000,0.001571,2.125000e-06,0.112656,2.141666e-06,8.333333e-09,0.99952,0.99952,0.007014,0.007014,2.732240e-03,4.098361e-03,8.333333e-09,0.403279,0.994604,7.861110e-07,8.333333e-09,0.000000,1.640819e-06,0.000000,0.000000e+00,8.333333e-09,0.000007,0.000007,8.333333e-09,0.000005,0.000005,8.333333e-09,0.091991,0.000000,1.0,0.000000,0.000000,0.000000,1.083333e-07,0.000000
3,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.004654,0.000000e+00,0.000000e+00,0.000000,0.000000,0.003891,0.000000,7.909416e-04,0.111921,7.909416e-04,0.000000e+00,0.99952,0.99952,0.000000,0.000000,3.512469e-06,5.268704e-06,0.000000e+00,0.400004,0.994604,7.909416e-04,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000003,0.000003,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,9.999999e-08,0.000000
4,0.316422,0.738485,0.390837,5.446429e-01,1.810760e-05,1.810760e-05,0.663912,0.663912,0.003922,1.535182e-05,1.535182e-05,0.667300,0.467164,0.003510,0.000806,5.408334e-01,0.111921,6.103317e-01,5.408333e-01,0.99952,0.99952,0.001319,0.001319,2.730756e-08,2.730756e-08,5.950000e-01,0.400000,0.994604,6.781473e-02,1.191667e-01,0.862973,2.523585e-01,0.000934,3.368678e-01,1.498765e-02,0.000014,0.000014,1.499569e-02,0.000023,0.000023,4.998564e-03,0.000000,0.540833,1.0,0.297500,0.054187,0.012447,6.249999e-07,0.000008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,0.003732,0.002363,0.004057,5.580357e-06,9.360354e-09,9.360354e-09,0.001373,0.001373,0.015472,1.500291e-05,1.500291e-05,0.000345,0.001249,0.000015,0.001249,3.250000e-07,0.112672,3.416666e-07,8.333333e-09,0.99952,0.99952,0.002578,0.002578,3.571429e-02,1.785714e-02,1.666667e-08,0.428571,0.994604,1.861111e-07,8.333333e-09,0.000000,1.702094e-07,0.002026,0.000000e+00,0.000000e+00,0.

In [ ]:
X40

,Average_Packet_Size,Packet_Length_Std,Packet_Length_Mean,Packet_Length_Variance,Subflow_Bwd_Bytes,Total_Length_of_Bwd_Packets,Avg_Bwd_Segment_Size,Bwd_Packet_Length_Mean,Init_Win_bytes_forward,Subflow_Fwd_Bytes,Total_Length_of_Fwd_Packets,Bwd_Packet_Length_Max,Max_Packet_Length,Init_Win_bytes_backward,Fwd_Packet_Length_Max,Flow_IAT_Max,Flow_Bytess,Flow_Duration,Fwd_IAT_Max,Fwd_Header_Length,Fwd_Header_Length.1,Avg_Fwd_Segment_Size,Fwd_Packet_Length_Mean,Fwd_Packetss,Bwd_Packetss,Fwd_IAT_Total,Flow_Packetss,Bwd_Header_Length,Flow_IAT_Mean,Fwd_IAT_Mean,Bwd_Packet_Length_Std,Flow_IAT_Std,Fwd_Packet_Length_Std,Fwd_IAT_Std,Bwd_IAT_Max,Subflow_Bwd_Packets,Total_Backward_Packets,Bwd_IAT_Total,Subflow_Fwd_Packets
0,0.022731,0.011808,0.021216,1.393393e-04,4.027749e-07,4.027749e-07,0.022757,0.022757,0.000000,4.661882e-06,4.651163e-06,0.006759,0.005318,0.000000,0.001209,2.624750e-04,0.111926,2.625333e-04,2.500000e-08,0.999305,0.999305,0.005051,0.005051,2.116402e-05,3.174603e-05,2.500000e-08,0.333354,0.966371,8.753333e-05,2.500000e-08,0.000000,2.143917e-04,0.000000,0.000000,3.333333e-08,0.000007,0.000007,3.333333e-08,0.000005
1,0.013613,0.001852,0.012705,3.428571e-06,1.586689e-07,1.586689e-07,0.008965,0.008965,0.000000,5.594259e-06,5.581395e-06,0.002663,0.002095,0.000000,0.001450,7.500000e-07,0.112338,1.541667e-06,4.000000e-07,0.999305,0.999305,0.006061,0.006061,3.683241e-03,5.524862e-03,4.000000e-07,0.337017,0.966371,5.361111e-07,4.000000e-07,0.000000,2.621880e-07,0.000000,0.000000,3.916667e-07,0.000007,0.000007,3.916667e-07,0.000005
2,0.030758,0.015811,0.028707,2.498527e-04,5.080456e-07,5.080456e-07,0.028704,0.028704,0.000000,7.536710e-06,7.519380e-06,0.011265,0.008864,0.000000,0.001974,6.170972e-01,0.111921,6.179400e-01,6.176828e-01,0.999305,0.999305,0.008166,0.008166,8.990446e-09,1.348567e-08,6.176828e-01,0.333333,0.966370,2.059800e-01,6.176828e-01,0.009233,5.038257e-01,0.000122,0.000000,6.173543e-01,0.000007,0.000007,6.173543e-01,0.000005
3,0.000771,0.000732,0.000599,5.357143e-07,9.153974e-09,9.153974e-09,0.001034,0.001034,0.445572,0.000000e+00,0.000000e+00,0.000307,0.000242,0.000015,0.000000,6.000000e-07,0.111959,6.000000e-07,0.000000e+00,0.999305,0.999305,0.000000,0.000000,4.901961e-03,7.352941e-03,0.000000e+00,0.338235,0.966370,6.000000e-07,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000003,0.000003,0.000000e+00,0.000000
4,0.002312,0.000000,0.001798,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000015,9.323764e-07,9.302326e-07,0.000000,0.000242,0.000000,0.000242,5.833333e-08,0.113636,5.833333e-08,2.500000e-08,0.999305,0.999305,0.001010,0.001010,2.222222e-01,0.000000e+00,2.500000e-08,0.444444,0.966370,5.833333e-08,2.500000e-08,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,0.003403,0.005009,0.003176,2.508036e-05,0.000000e+00,0.000000e+00,0.000000,0.000000,0.948685,4.117996e-06,4.108527e-06,0.000000,0.002135,0.556961,0.002135,2.111718e-02,0.111921,2.141082e-02,2.141038e-02,0.999305,0.999305,0.002975,0.002975,3.892118e-07,1.946059e-07,2.141079e-02,0.333334,0.966370,7.136964e-03,1.070540e-02,0.000000,1.713407e-02,0.005279,0.021473,0.000000e+00,0.000003,0.000003,0.000000e+00,0.000009
999996,0.011045,0.001852,0.010308,3.428571e-06,1.342583e-07,1.342583e-07,0.007586,0.007586,0.000000,4.351090e-06,4.341085e-06,0.002253,0.001773,0.000000,0.001128,1.125000e-06,0.112369,1.183333e-06,3.333333e-08,0.999305,0.999305,0.004714,0.004714,4.830918e-03,7.246377e-03,3.333333e-08,0.338164,0.966371,4.166667e-07,3.333333e-08,0.000000,8.680853e-07,0.000000,0.000000,2.500000e-08,0.000007,0.000007,2.500000e-08,0.000005
999997,0.007384,0.000000,0.006892,0.000000e+00,7.018047e-08,7.018047e-08,0.003965,0.003965,0.000000,3.574110e-06,3.565891e-06,0.001178,0.000927,0.000000,0.000927,8.916666e-07,0.112119,1.691667e-0

In [ ]:
#saving dataframe to csv in the drive
X40.to_csv(r'drive/MyDrive/NS/Lab/MachineLearningCSV/CSVs/X40_prepro.csv')

In [ ]:
#saving dataframe to csv in the drive
X50.to_csv(r'drive/MyDrive/NS/Lab/MachineLearningCSV/CSVs/X50_prepro.csv')

In [ ]:
print(y1.index.name)

None


In [ ]:
y1.head()

,Label
0,0
1,0
2,0
3,0
4,7


In [ ]:
y1.rename(columns={0:'Label'}, inplace=True)

In [ ]:
y1.nunique()

Label    15
dtype: int64

In [ ]:
y1.shape

(998984, 1)

In [ ]:
y1.to_csv(r'drive/MyDrive/NS/Lab/MachineLearningCSV/CSVs/y_prepro.csv')

In [ ]:
# Train Test Split
x_train, x_test, y_train, y_test = train_test_split(X, y1, test_size=0.3, random_state=101)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((699295, 77), (699295, 1), (299698, 77), (299698, 1))

In [ ]:
X

,Flow_Duration,Total_Fwd_Packets,Total_Backward_Packets,Total_Length_of_Fwd_Packets,Total_Length_of_Bwd_Packets,Fwd_Packet_Length_Max,Fwd_Packet_Length_Min,Fwd_Packet_Length_Mean,Fwd_Packet_Length_Std,Bwd_Packet_Length_Max,Bwd_Packet_Length_Min,Bwd_Packet_Length_Mean,Bwd_Packet_Length_Std,Flow_Bytess,Flow_Packetss,Flow_IAT_Mean,Flow_IAT_Std,Flow_IAT_Max,Flow_IAT_Min,Fwd_IAT_Total,Fwd_IAT_Mean,Fwd_IAT_Std,Fwd_IAT_Max,Fwd_IAT_Min,Bwd_IAT_Total,Bwd_IAT_Mean,Bwd_IAT_Std,Bwd_IAT_Max,Bwd_IAT_Min,Fwd_PSH_Flags,Bwd_PSH_Flags,Fwd_URG_Flags,Bwd_URG_Flags,Fwd_Header_Length,Bwd_Header_Length,Fwd_Packetss,Bwd_Packetss,Min_Packet_Length,Max_Packet_Length,Packet_Length_Mean,Packet_Length_Std,Packet_Length_Variance,FIN_Flag_Count,SYN_Flag_Count,RST_Flag_Count,PSH_Flag_Count,ACK_Flag_Count,URG_Flag_Count,CWE_Flag_Count,ECE_Flag_Count,DownUp_Ratio,Average_Packet_Size,Avg_Fwd_Segment_Size,Avg_Bwd_Segment_Size,Fwd_Header_Length.1,Fwd_Avg_BytesBulk,Fwd_Avg_PacketsBulk,Fwd_Avg_Bulk_Rate,Bwd_Avg_BytesBulk,Bwd_Avg_PacketsBulk,Bwd_Avg_Bulk_Rate,Subflow_Fwd_Packets,Subflow_Fwd_Bytes,Subflow_Bwd_Packets,Subflow_Bwd_Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active_Mean,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min
0,156,2,0,12,0,6,6,6.000000,0.000000,0,0,0.000000,0.000000,7.692308e+04,12820.512820,1.560000e+02,0.000000e+00,156,156,156,156.000,0.000000e+00,156,156,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,40,0,12820.512820,0.000000,6,6,6.000000,0.000000,0.000000,0,0,0,0,1,0,0,0,0,9.000000,6.000000,0.000000,40,0,0,0,0,0,0,2,12,0,0,256,-1,1,20,0.0,0.000,0,0,0.0,0.0,0,0
1,85365947,20,20,1618,6190,427,0,80.900000,117.724880,1410,0,309.500000,464.204639,9.146504e+01,0.468571,2.188870e+06,1.010000e+07,58500000,1,85400000,4492944.579,1.430000e+07,58500000,26,85300000,4.488439e+06,1.430000e+07,58600000,1,0,0,0,0,424,412,0.234285,0.234285,0,1410,190.439024,350.617673,122932.752400,0,0,0,1,0,0,0,0,1,195.200000,80.900000,309.500000,424,0,0,0,0,0,0,20,1618,20,6190,8192,132,18,20,620266.5,756234.439,1155005,85528,42000000.0,23400000.0,58500000,25500000
2,150,2,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000e+00,13333.333330,1.500000e+02,0.000000e+00,150,150,150,150.000,0.000000e+00,150,150,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,40,0,13333.333330,0.000000,0,0,0.000000,0.000000,0.000000,0,0,0,0,1,0,0,0,0,0.000000,0.000000,0.000000,40,0,0,0,0,0,0,2,0,0,0,255,-1,0,20,0.0,0.000,0,0,0.0,0.0,0,0
3,3,2,0,4,0,2,2,2.000000,0.000000,0,0,0.000000,0.000000,1.333333e+06,666666.666700,3.000000e+00,0.000000e+00,3,3,3,3.000,0.000000e+00,3,3,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,48,0,666666.666700,0.000000,2,2,2.000000,0.000000,0.000000,0,0,0,1,0,0,0,0,0,3.000000,2.000000,0.000000,48,0,0,0,0,0,0,2,4,0,0,1024,-1,1,24,0.0,0.000,0,0,0.0,0.0,0,0
4,8584262,9,15,105,188,23,0,11.666667,9.354143,34,0,12.533333,14.549750,3.413223e+01,2.795814,3.732288e+05,9.772724e+05,3143363,3,6319547,789943.375,1.456589e+06,3189621,4,8584140,6.131529e+05,1.207415e+06,3143363,3,0,0,0,0,296,488,1.048430,1.747384,0,34,11.720000,12.601323,158.793333,0,0,0,1,0,0,0,0,1,12.208333,11.666667,12.533333,296,0,0,0,0,0,0,9,105,15,188,29200,227,6,32,0.0,0.000,0,0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,5566006,3,1,12,0,6,0,4.000000,3.464102,0,0,0.000000,0.000000,2.155944e+00,0.718648,1.855335e+06,3.169759e+06,5515342,71,5566006,2783003.000,3.864111e+06,5515342,50664,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,72,32,0.538986,0.179662,0,6,2.400000,3.286335,10.800000,0,0,0,1,0,0,0,0,0,3.000000,4.000000,0.000000,72,0,0,0,0,0,0,3,12,1,0,8192,26883,2,20,0.0,0.000,0,0,0.0,0.0,0,0
999996,464,1,1,48,108,48,48,48.000000,0.000000,108,108,108.000000,0.000000,3.362069e+0

In [ ]:
x_train.head(5)

,Flow_Duration,Total_Fwd_Packets,Total_Backward_Packets,Total_Length_of_Fwd_Packets,Total_Length_of_Bwd_Packets,Fwd_Packet_Length_Max,Fwd_Packet_Length_Min,Fwd_Packet_Length_Mean,Fwd_Packet_Length_Std,Bwd_Packet_Length_Max,Bwd_Packet_Length_Min,Bwd_Packet_Length_Mean,Bwd_Packet_Length_Std,Flow_Bytess,Flow_Packetss,Flow_IAT_Mean,Flow_IAT_Std,Flow_IAT_Max,Flow_IAT_Min,Fwd_IAT_Total,Fwd_IAT_Mean,Fwd_IAT_Std,Fwd_IAT_Max,Fwd_IAT_Min,Bwd_IAT_Total,Bwd_IAT_Mean,Bwd_IAT_Std,Bwd_IAT_Max,Bwd_IAT_Min,Fwd_PSH_Flags,Bwd_PSH_Flags,Fwd_URG_Flags,Bwd_URG_Flags,Fwd_Header_Length,Bwd_Header_Length,Fwd_Packetss,Bwd_Packetss,Min_Packet_Length,Max_Packet_Length,Packet_Length_Mean,Packet_Length_Std,Packet_Length_Variance,FIN_Flag_Count,SYN_Flag_Count,RST_Flag_Count,PSH_Flag_Count,ACK_Flag_Count,URG_Flag_Count,CWE_Flag_Count,ECE_Flag_Count,DownUp_Ratio,Average_Packet_Size,Avg_Fwd_Segment_Size,Avg_Bwd_Segment_Size,Fwd_Header_Length.1,Fwd_Avg_BytesBulk,Fwd_Avg_PacketsBulk,Fwd_Avg_Bulk_Rate,Bwd_Avg_BytesBulk,Bwd_Avg_PacketsBulk,Bwd_Avg_Bulk_Rate,Subflow_Fwd_Packets,Subflow_Fwd_Bytes,Subflow_Bwd_Packets,Subflow_Bwd_Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active_Mean,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min
269778,5.257048e-01,0.000027,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005761,0.333333,8.750010e-02,1.403302e-01,2.666667e-01,8.307332e-03,5.258333e-01,8.750000e-02,0.140965,2.666667e-01,8.307225e-03,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.999856,0.994592,3.698739e-08,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.999856,0.0,0.0,0.0,0.0,0.0,0.0,0.000027,0.000000,0.000000,0.000000e+00,0.445572,0.00000,0.000000,1.0,0.063680,0.000000,0.063680,0.063680,0.155833,0.159269,0.266667,0.066799
788774,4.333333e-07,0.000005,0.000007,0.000007,2.593626e-07,0.001853,0.022276,0.008389,0.000000,0.005870,0.058219,0.014654,0.000000,0.008986,0.350427,2.166666e-07,2.249857e-07,4.000000e-07,1.250000e-07,8.333333e-09,8.333333e-09,0.000000,8.333333e-09,1.666667e-08,2.500000e-08,2.500000e-08,0.000000,2.500000e-08,2.500000e-08,0.0,0.0,0.0,0.0,0.999856,0.994592,1.709402e-02,2.564103e-02,0.031768,0.003425,0.021096,0.004515,0.000020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.009259,0.023074,0.008389,0.014654,0.999856,0.0,0.0,0.0,0.0,0.0,0.0,0.000005,0.000007,0.000007,2.593626e-07,0.000000,0.00000,0.000005,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
277696,1.550000e-06,0.000005,0.000007,0.000006,2.715679e-07,0.001491,0.017918,0.006747,0.000000,0.006146,0.060959,0.015344,0.000000,0.006460,0.337187,5.888888e-07,7.319241e-07,1.141667e-06,1.250000e-07,8.333333e-09,8.333333e-09,0.000000,8.333333e-09,1.666667e-08,4.000000e-07,4.000000e-07,0.000000,4.000000e-07,4.000000e-07,0.0,0.0,0.0,0.0,0.999856,0.994592,3.853565e-03,5.780347e-03,0.025552,0.003586,0.019795,0.006020,0.000036,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.009259,0.021650,0.006747,0.015344,0.999856,0.0,0.0,0.0,0.0,0.0,0.0,0.000005,0.000006,0.000007,2.715679e-07,0.000000,0.00000,0.000005,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
76662,9.502915e-01,0.000086,0.000065,0.000260,2.175594e-06,0.060999,0.000000,0.030528,0.060401,0.043785,0.000000,0.012939,0.024279,0.005761,0.333333,2.500777e-02,5.377836e-02,8.333343e-02,1.500000e-07,9.500000e-01,4.999924e-02,0.058206,8.416667e-02,7.558333e-06,9.500000e-01,5.277701e-02,0.057380,8.416667e-02,3.333333e-08,0.0,0.0,0.0,0.0,0.999856,0.994593,5.846160e-08,8.330778e-08,0.000000,0.060999,0.040873,0.070420,0.004956,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.036681,0.030528,0.012939,0.999856,0.0,0.0,0.0,0.0,0.0,0.0,0.000086,0.000260,0.000065,2.175594e-06,0.445572,0.00618,0.000019,1.0,0.003232,0.013632,0.029455,0.000332,0.083333,0.000217,0.083333,0.083001
913025,1.336459e-02,0.000009,0.000

#### running random forest and decision tree

In [ ]:
models=[RandomForestClassifier(), DecisionTreeClassifier()]
model_names=['RandomForestClassifier','DecisionTree']
acc=[]
d={}
for model in range(len(models)):
    clf=models[model]
    clf.fit(x_train,y_train)
    pred=clf.predict(x_test)
    acc.append(accuracy_score(pred,y_test))

d={'Modelling Algo':model_names,'Accuracy':acc}

In [ ]:
acc_frame=pd.DataFrame(d)
acc_frame

In [ ]:
param_grid = {
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}
CV_rfc = GridSearchCV(estimator=RandomForestClassifier(), param_grid=param_grid, scoring='accuracy', cv= 5)
CV_rfc.fit(x_train, y_train)

In [ ]:
print("Best score : ",CV_rfc.best_score_)
print("Best Parameters : ",CV_rfc.best_params_)
print("Precision Score : ", precision_score(CV_rfc.predict(x_test),y_test))

In [ ]:
df1 = pd.DataFrame.from_records(x_train)
tmp = pd.DataFrame({'Feature': df1.columns, 'Feature importance': clf_rf.feature_importances_})
tmp = tmp.sort_values(by='Feature importance',ascending=False)
plt.figure(figsize = (7,4))
plt.title('Features importance',fontsize=14)
s = sns.barplot(x='Feature',y='Feature importance',data=tmp)
s.set_xticklabels(s.get_xticklabels(),rotation=90)
plt.show()

#### running models
random forest

In [ ]:
# let us create and run different models on the dataset
# let us use randomforest
# import libraries
from sklearn.ensemble import RandomForestClassifier
ml=RandomForestClassifier(n_estimators=100)
import time
stime = time.time()
ml.fit(X_train,y_train)
print("--- %s seconds needed to train ---" % (time.time() - stime))
y_pred=ml.predict(X_test)
y_pred=pd.DataFrame(y_pred.reshape(-1,1))
y_pred.head(4)

--- 350.4873425960541 seconds needed to train ---


,0
0,BENIGN
1,BENIGN
2,BENIGN
3,BENIGN


In [ ]:
# from sklearn.preprocessing import LabelEncoder
# le= LabelEncoder()
# y_test=le.fit_transform(y_test)
# y_test

In [ ]:
# let us check accuracy
# import metrics library
from sklearn import metrics
print("accuracy is: ", metrics.accuracy_score(y_test,y_pred))
# import confusion matrix
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test,y_pred))
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
print("f1 score: ", f1_score(y_test,y_pred,average='weighted'))
print("precision score: ", precision_score(y_test, y_pred, average='weighted'))
print("recall score: ", recall_score(y_test, y_pred, average='weighted'))

accuracy is:  0.9975874587899254
[[240451     21      8      4     62     35      0      8      0      0
     101      0      0      0      0]
 [   101     94      0      0      0      0      0      0      0      0
       0      0      0      0      0]
 [    19      0  13487      0      2      0      0      0      0      0
       0      0      0      0      0]
 [     5      0      0   1076      4      0      0      0      0      0
       0      0      0      0      0]
 [   217      0      0      4  24288      0      0      0      0      0
       2      0      0      0      0]
 [     7      0      0      0      0    580      1      0      0      0
       0      0      1      0      0]
 [     0      0      0      0      0      2    637      0      0      0
       0      0      0      0      0]
 [     1      0      0      0      0      0      0    813      0      0
       0      0      0      0      0]
 [     0      0      0      0      0      0      0      0      2      0
       0      0

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


precision score:  0.9974965703000676
recall score:  0.9975874587899254


##### Ada boost algorithm

In [ ]:
# let us create and run different models on the dataset
# let us use randomforest
# import libraries
from sklearn.ensemble import AdaBoostClassifier
clf=AdaBoostClassifier(random_state=96)
import time
stime = time.time()
clf.fit(x_train,y_train)
print("--- %s seconds needed to train ---" % (time.time() - stime))
print("training score: ",clf.score(x_train, y_train))
y_pred=clf.predict(x_test)
y_pred=pd.DataFrame(y_pred.reshape(-1,1))
y_pred.head(4)

--- 175.72682547569275 seconds needed to train ---
training score:  0.8571103738237565


,0
0,0
1,0
2,0
3,0


In [ ]:
# let us check accuracy
# import metrics library
from sklearn import metrics
print("accuracy is: ", metrics.accuracy_score(y_test,y_pred))
# import confusion matrix
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test,y_pred))
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
print("f1 score: ", f1_score(y_test,y_pred,average='weighted'))
print("precision score: ", precision_score(y_test, y_pred, average='weighted'))
print("recall score: ", recall_score(y_test, y_pred, average='weighted'))

accuracy is:  0.8562113967091893
[[240145      0      0      0    281      0      0      0      0      0
       0      0      0      0      0]
 [   215      0      0      0      0      0      0      0      0      0
       0      0      0      0      0]
 [  5008      0      0      0   8461      0      0      0      0      0
       0      0      0      0      0]
 [   370      0      0      0    778      0      0      0      0      0
       0      0      0      0      0]
 [  7962      0      0      0  16445      0      0      0      0      0
       0      0      0      0      0]
 [   548      0      0      0      0      0      0      0      0      0
       0      0      0      0      0]
 [   621      0      0      0      0      0      0      0      0      0
       0      0      0      0      0]
 [   878      0      0      0      0      0      0      0      0      0
       0      0      0      0      0]
 [     0      0      0      0      0      0      0      0      2      0
       0      0

#### stacking ensemble classifier

In [ ]:
X

,Flow_Duration,Total_Fwd_Packets,Total_Backward_Packets,Total_Length_of_Fwd_Packets,Total_Length_of_Bwd_Packets,Fwd_Packet_Length_Max,Fwd_Packet_Length_Min,Fwd_Packet_Length_Mean,Fwd_Packet_Length_Std,Bwd_Packet_Length_Max,Bwd_Packet_Length_Min,Bwd_Packet_Length_Mean,Bwd_Packet_Length_Std,Flow_Bytess,Flow_Packetss,Flow_IAT_Mean,Flow_IAT_Std,Flow_IAT_Max,Flow_IAT_Min,Fwd_IAT_Total,Fwd_IAT_Mean,Fwd_IAT_Std,Fwd_IAT_Max,Fwd_IAT_Min,Bwd_IAT_Total,Bwd_IAT_Mean,Bwd_IAT_Std,Bwd_IAT_Max,Bwd_IAT_Min,Fwd_PSH_Flags,Bwd_PSH_Flags,Fwd_URG_Flags,Bwd_URG_Flags,Fwd_Header_Length,Bwd_Header_Length,Fwd_Packetss,Bwd_Packetss,Min_Packet_Length,Max_Packet_Length,Packet_Length_Mean,Packet_Length_Std,Packet_Length_Variance,FIN_Flag_Count,SYN_Flag_Count,RST_Flag_Count,PSH_Flag_Count,ACK_Flag_Count,URG_Flag_Count,CWE_Flag_Count,ECE_Flag_Count,DownUp_Ratio,Average_Packet_Size,Avg_Fwd_Segment_Size,Avg_Bwd_Segment_Size,Fwd_Header_Length.1,Fwd_Avg_BytesBulk,Fwd_Avg_PacketsBulk,Fwd_Avg_Bulk_Rate,Bwd_Avg_BytesBulk,Bwd_Avg_PacketsBulk,Bwd_Avg_Bulk_Rate,Subflow_Fwd_Packets,Subflow_Fwd_Bytes,Subflow_Bwd_Packets,Subflow_Bwd_Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active_Mean,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min
0,5.083333e-07,0.000000,0.000003,1.550388e-07,9.153974e-09,0.000081,0.000969,0.000365,0.000000,0.000414,0.004110,0.001034,0.000000,0.005841,0.340278,5.083333e-07,0.000000e+00,5.083333e-07,5.166666e-07,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,8.333333e-09,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.999856,0.994592,6.944444e-03,1.041667e-02,0.001381,0.000242,0.001142,0.000488,2.380952e-07,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.009259,0.001498,0.000365,0.001034,0.999856,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.553961e-07,0.000003,9.153974e-09,0.015640,0.000015,0.000000,1.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
1,7.061107e-01,0.000032,0.000017,2.790698e-05,1.769005e-05,0.014263,0.000000,0.008206,0.017524,0.500000,0.000000,0.399793,0.406562,0.005761,0.333333,5.884265e-02,2.877358e-01,7.050000e-01,1.250000e-07,7.050000e-01,1.008333e-01,0.379066,7.050000e-01,1.666667e-08,1.288567e-03,3.221417e-04,0.000806,1.168558e-03,1.583333e-06,0.0,0.0,0.0,0.0,0.999856,0.994592,3.147130e-08,2.950435e-08,0.000000,0.291700,0.292441,0.458585,2.101810e-01,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.275570,0.008206,0.399793,0.999856,0.0,0.0,0.0,0.0,0.0,0.0,0.000032,2.797129e-05,0.000017,1.769005e-05,0.004196,0.003601,0.000009,1.0,0.000018,0.0,0.000018,0.000018,0.705000,0.000000,0.70500,0.705000
2,5.042358e-03,0.000000,0.000003,4.263566e-06,8.391143e-08,0.002216,0.026634,0.010030,0.000000,0.003798,0.037671,0.009482,0.000000,0.005761,0.333334,5.042358e-03,0.000000e+00,5.042358e-03,5.042366e-03,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,8.333333e-09,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.999856,0.994592,5.509004e-07,8.263507e-07,0.037983,0.002216,0.018836,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.009259,0.024722,0.010030,0.009482,0.999856,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,4.273392e-06,0.000003,8.391143e-08,0.000000,0.000000,0.000000,1.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
3,8.447685e-02,0.000032,0.000034,1.117054e-04,8.983100e-06,0.044239,0.000000,0.032848,0.053025,0.116644,0.001370,0.101508,0.087787,0.005761,0.333334,4.969328e-03,2.857536e-02,8.332960e-02,1.166667e-07,1.147233e-03,1.638905e-04,0.000243,4.373083e-04,3.333333e-08,8.420777e-02,9.356419e-03,0.039433,8.333027e-02,1.666667e-08,0.0,0.0,0.0,0.0,0.999856,0.994592,2.630573e-07,4.932324e-07,0.000000,0.068050,0.132102,0.127553,1.626058e-02,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.009259,0.122011,0.032848,0.101508,0.999856,0.0,0.0,0.0,0.0,0.0,0.0,0.000032,1.119629e-04,0.000034,8.983100e-06,0.125015,0.485886,0.000033,1.0,0.001252,0.0,0.001252,0.001252,0.083330,0.000000,0.08333,0.083330
4,3.234167e-04,0.000000,0

In [ ]:
y1

,0
0,10
1,4
2,0
3,0
4,0
...,...
998948,0
998949,2
998950,0
998951,0


In [ ]:
# compare ensemble to each baseline classifier
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import StackingClassifier
from matplotlib import pyplot

In [ ]:
# get a stacking ensemble of models
def get_stacking():
	# define the base models
	level0 = list()
	level0.append(('lr', LogisticRegression()))
	level0.append(('knn', KNeighborsClassifier()))
	level0.append(('cart', DecisionTreeClassifier()))
	level0.append(('svm', SVC()))
	level0.append(('bayes', GaussianNB()))
	# define meta learner model
	level1 = LogisticRegression()
	# define the stacking ensemble
	model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)
	return model

In [ ]:
# get a list of models to evaluate
def get_models():
	models = dict()
	models['lr'] = LogisticRegression()
	models['knn'] = KNeighborsClassifier()
	models['cart'] = DecisionTreeClassifier()
	models['svm'] = SVC()
	models['bayes'] = GaussianNB()
	models['stacking'] = get_stacking()
	return models

In [ ]:
# define dataset
#X, y = data[:, :-1], data[:, -1]
X, y = X, y1
# get the models to evaluate
models = get_models()

In [ ]:

# get the dataset
# def get_dataset():
# 	X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=1)
# 	return X, y

# evaluate a give model using cross-validation
def evaluate_model(model, X, y):
	cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
	scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
	return scores

In [ ]:
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
	scores = evaluate_model(model, X, y)
	results.append(scores)
	names.append(name)
	print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))


>lr 0.926 (0.002)


KeyboardInterrupt: ignored

In [ ]:
# plot model performance for comparison
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.show()

#### voting ensemble classifier

In [ ]:
# compare hard voting to standalone classifiers
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from matplotlib import pyplot

# get the dataset
def get_dataset():
	X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=2)
	return X, y

# get a voting ensemble of models
def get_voting():
	# define the base models
	models = list()
	models.append(('knn1', KNeighborsClassifier(n_neighbors=1)))
	models.append(('knn3', KNeighborsClassifier(n_neighbors=3)))
	models.append(('knn5', KNeighborsClassifier(n_neighbors=5)))
	models.append(('knn7', KNeighborsClassifier(n_neighbors=7)))
	models.append(('knn9', KNeighborsClassifier(n_neighbors=9)))
	# define the voting ensemble
	ensemble = VotingClassifier(estimators=models, voting='hard')
	return ensemble

# get a list of models to evaluate
def get_models():
	models = dict()
	models['knn1'] = KNeighborsClassifier(n_neighbors=1)
	models['knn3'] = KNeighborsClassifier(n_neighbors=3)
	models['knn5'] = KNeighborsClassifier(n_neighbors=5)
	models['knn7'] = KNeighborsClassifier(n_neighbors=7)
	models['knn9'] = KNeighborsClassifier(n_neighbors=9)
	models['hard_voting'] = get_voting()
	return models

# evaluate a give model using cross-validation
def evaluate_model(model, X, y):
	cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
	scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
	return scores

# define dataset
X, y = get_dataset()
# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
	scores = evaluate_model(model, X, y)
	results.append(scores)
	names.append(name)
	print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))
# plot model performance for comparison
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.show()